In [1]:
import numpy as np
import pandas as pd
import scipy.optimize
import bokeh.plotting
import bokeh.io
from bokeh.io import export_png
from bokeh.plotting import curdoc 
from natsort import natsorted
from matplotlib import pyplot as plt
import glob

bokeh.io.output_notebook()

Loading BokehJS ...

In [2]:
def agonist_only_p_active_theory(c_agonist, log_Kd_active, log_Kd_inactive, beta_deltaE):
    "theoretical curve for mglur5 active probability"

    Kd_active = 10**(log_Kd_active)
    Kd_inactive = 10**(log_Kd_inactive)
    a = (1 + c_agonist/Kd_active)

    b = (1 + c_agonist/Kd_inactive)

    return a/(a+b*np.exp(-beta_deltaE))

def resid(param, c, p_active):
    return p_active - agonist_only_p_active_theory(c, *param)



In [3]:
#set up plot
from bokeh.plotting import curdoc 
curdoc().clear()
p = bokeh.plotting.figure(plot_height=600,
                         plot_width=600,
                         x_axis_label='[Ligands]M',
                          y_axis_label='active_probability',
                          x_axis_type='log')

colors=bokeh.palettes.Magma[11]

ligand_smooth=np.logspace(-13,-5,200)

for i,deltaE in enumerate(np.linspace(-1,-4,9)):    
    p.line(ligand_smooth, agonist_only_p_active_theory(ligand_smooth, -10.9, -9.9, deltaE), color=colors[i],legend_label='delta E='+str(deltaE))

p.legend.location = "top_left"

bokeh.io.show(p);

In [4]:
#set up plot
from bokeh.plotting import curdoc 
curdoc().clear()
p = bokeh.plotting.figure(plot_height=600,
                         plot_width=600,
                         x_axis_label='[Ligands]M',
                          y_axis_label='active_probability',
                          x_axis_type='log')

colors=bokeh.palettes.Magma[11]
ligand_smooth=np.logspace(-13,-5,200)

for i,Kd_inactive in enumerate(np.linspace(-10,-8,9)):    
    p.line(ligand_smooth, agonist_only_p_active_theory(ligand_smooth, -10.9, Kd_inactive, -1), color=colors[i],legend_label='log_Kd_inactive='+str(Kd_inactive))

p.legend.location = "top_left"

bokeh.io.show(p);

In [5]:
#set up plot

curdoc().clear()
p = bokeh.plotting.figure(plot_height=600,
                         plot_width=600,
                         x_axis_label='[Ligands]M',
                          y_axis_label='active_probability',
                          x_axis_type='log')



for i,Kd_active in enumerate(np.linspace(-11,-8,9)):    
    p.line(ligand_smooth, agonist_only_p_active_theory(ligand_smooth, Kd_active, -9, -1), color=colors[i],legend_label='log_Kd_active='+str(Kd_active))

p.legend.location = "top_left"

bokeh.io.show(p);

In [12]:
#set up plot

curdoc().clear()
p = bokeh.plotting.figure(plot_height=600,
                         plot_width=900,
                         x_axis_label='[Ligands]M',
                          y_axis_label='active_probability',
                          x_axis_type='log')



for i,Kd_active in enumerate(np.linspace(-11,-8,9)):    
    Kd_inactive = Kd_active+2
    p.line(ligand_smooth, agonist_only_p_active_theory(ligand_smooth, Kd_active, Kd_inactive, -1), color=colors[i],legend_label='log_Kd_active='+str(Kd_active)+', log_Kd_active='+str(Kd_inactive))

p.legend.location = "top_left"

bokeh.io.show(p);

In [6]:
fnames = natsorted(glob.glob("./jiawei_trupath_csv/MOR_*_Gi1_normalize_deer+Gi.xlsx"))
fnames

['./jiawei_trupath_csv/MOR_BU72_Gi1_normalize_deer+Gi.xlsx',
 './jiawei_trupath_csv/MOR_DAMGO_Gi1_normalize_deer+Gi.xlsx',
 './jiawei_trupath_csv/MOR_MP_Gi1_normalize_deer+Gi.xlsx',
 './jiawei_trupath_csv/MOR_PZM21_Gi1_normalize_deer+Gi.xlsx',
 './jiawei_trupath_csv/MOR_TRV130_Gi1_normalize_deer+Gi.xlsx',
 './jiawei_trupath_csv/MOR_buprenorphine_Gi1_normalize_deer+Gi.xlsx',
 './jiawei_trupath_csv/MOR_lofentanil_Gi1_normalize_deer+Gi.xlsx',
 './jiawei_trupath_csv/MOR_morphine_Gi1_normalize_deer+Gi.xlsx']

In [7]:
curdoc().clear()
p = bokeh.plotting.figure(plot_height=600,
                         plot_width=600,
                         x_axis_label='[Ligands] M',
                          y_axis_label='active_probability',
                          x_axis_type='log')
colors=bokeh.palettes.d3['Category10'][10]

for i,ligand in enumerate(df.columns[1:]):
    c, p_active = df['conc'].values, df[ligand].values
    p.circle(c, p_active, size=7, color=colors[i],legend_label=ligand)

bokeh.io.show(p);

NameError: name 'df' is not defined

In [8]:
import matplotlib
matplotlib.rc('font', size=20)

In [90]:
fnames = natsorted(glob.glob("./jiawei_trupath_csv/MOR_*_Gi1_normalize_deer+Gi.xlsx"))
kd_table = pd.DataFrame(columns = ["ligand","Kd_active (nM)", "Kd_inactive (nM)", "beta_deltaE"])

for file in fnames:
    
    df = pd.read_excel(file)

    #initial parameter for optimization
    p0 = np.array([-7,-4,-4.5])
    ligand_name = df.columns[1]
    c, p_active = df['conc'].values, df[ligand_name].values

    #optimize kd and energy difference to fit the data
    res = scipy.optimize.least_squares(resid, p0, args = (c,p_active))

    plt.figure(figsize=(15, 9))
    plt.rcParams['font.size'] = '30'
    plt.scatter(c, p_active)
    plt.plot(ligand_smooth, agonist_only_p_active_theory(ligand_smooth, res.x[0],res.x[1],res.x[2]), 'r')
    plt.xlabel(ligand_name+' [M]')
    plt.ylabel('active_probability')
    plt.ylim([0, 1])
    plt.xscale('log')
    plt.xticks([1e-13,1e-12,1e-11,1e-10,1e-9,1e-8,1e-7,1e-6,1e-5,1e-4])
    #plt.legend("Kd_active={}, Kd_inactive={}, beta_deltaE={}".format(10**res.x[0],10**res.x[1],res.x[2]))
    #plt.figtext(0.2, 0.7, "Kd_active={}, Kd_inactive={}, beta_deltaE={}".format(10**res.x[0],10**res.x[1],res.x[2]), size='large') 

    plt.grid()
    plt.savefig('{}_Gi_fitting.png'.format(ligand_name),dpi=600)  

    """
    p = bokeh.plotting.figure(plot_height=400,
                             plot_width=600,
                             x_axis_label=ligand_name+' [M]',
                              y_axis_label='active_probability',
                              x_axis_type='log')

    p.line(ligand_smooth, agonist_only_p_active_theory(ligand_smooth, res.x[0],res.x[1],res.x[2]))
    p.circle(c, p_active, size=7)


    bokeh.io.show(p);
    """
    res.x = np.around(res.x,2)
    kd_table.loc[len(kd_table)] = [ligand_name,np.around(10**res.x[0]*1e9,2),np.around(10**res.x[1]*1e9,2),np.around(res.x[2],2)]

kd_table

,ligand,Kd_active (nM),Kd_inactive (nM),beta_deltaE
0,BU72,0.07,0.78,-1.08
1,damgo,2.82,21.38,-1.13
2,MP,0.30,3.16,-1.12
3,PZM21,2.75,13.18,-1.06
4,TRV,4.68,19.05,-1.10
5,buprenorphine,0.16,0.98,-1.15
6,lofentanil,0.01,0.12,-1.04
7,morphine,0.63,3.16,-1.05


In [91]:
kd_table.sort_values(by=['Kd_active (nM)'])

,ligand,Kd_active (nM),Kd_inactive (nM),beta_deltaE
6,lofentanil,0.01,0.12,-1.04
0,BU72,0.07,0.78,-1.08
5,buprenorphine,0.16,0.98,-1.15
2,MP,0.30,3.16,-1.12
7,morphine,0.63,3.16,-1.05
3,PZM21,2.75,13.18,-1.06
1,damgo,2.82,21.38,-1.13
4,TRV,4.68,19.05,-1.10


In [27]:
kd_table.sort_values(by=['Kd_inactive (nM)'])

,ligand,Kd_active (nM),Kd_inactive (nM),delta_E
6,lofentanil,0.01,0.12,-1.04
0,BU72,0.07,0.78,-1.08
5,buprenorphine,0.16,0.98,-1.15
2,MP,0.30,3.16,-1.12
7,morphine,0.63,3.16,-1.05
3,PZM21,2.75,13.18,-1.06
4,TRV,4.68,19.05,-1.10
1,damgo,2.82,21.38,-1.13


In [40]:
df = pd.read_excel('./jiawei_trupath_csv/MOR_DAMGO_Gi1_normalize_deer+Gi.xlsx')
curdoc().clear()
#initial parameter for optimization
p0 = np.array([-7,-4,-4.5])
ligand_name = df.columns[1]
c, p_active = df['conc'].values, df[ligand_name].values

#optimize kd and energy difference to fit the data
popt,pcov = scipy.optimize.curve_fit(agonist_only_p_active_theory, c, p_active)

curdoc().clear()
p = bokeh.plotting.figure(plot_height=400,
                         plot_width=600,
                         x_axis_label=ligand_name+' [M]',
                          y_axis_label='active_probability',
                          x_axis_type='log')

p.line(ligand_smooth, agonist_only_p_active_theory(ligand_smooth, popt[0],popt[1],popt[2]))
p.circle(c, p_active, size=7)

popt

<ipython-input-2-601712ba33c4>:6: RuntimeWarning: divide by zero encountered in true_divide
  a = (1 + c_agonist/Kd_active)
<ipython-input-2-601712ba33c4>:8: RuntimeWarning: divide by zero encountered in true_divide
  b = (1 + c_agonist/Kd_inactive)
<ipython-input-2-601712ba33c4>:10: RuntimeWarning: invalid value encountered in true_divide
  return a/(a+b*np.exp(-beta_deltaE))
<ipython-input-2-601712ba33c4>:5: RuntimeWarning: overflow encountered in double_scalars
  Kd_inactive = 10**(log_Kd_inactive)
/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/minpack.py:828: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


array([14.21456734,  9.16928304, -0.1896139 ])

In [35]:
fnames = natsorted(glob.glob("./jiawei_trupath_csv/MOR_*_arr1_normalize_deer+arr.xlsx"))
kd_table_arr = pd.DataFrame(columns = ["ligand","Kd_active (nM)", "Kd_inactive (nM)", "delta_E"])

for file in fnames:
    
    df = pd.read_excel(file)

    #initial parameter for optimization
    p0 = np.array([-7,-4,-4.5])
    ligand_name = df.columns[1]
    c, p_active = df['conc'].values, df[ligand_name].values

    #optimize kd and energy difference to fit the data
    res = scipy.optimize.least_squares(resid, p0, args = (c,p_active))

    plt.figure(figsize=(15, 9))

    plt.scatter(c, p_active)
    plt.plot(ligand_smooth, agonist_only_p_active_theory(ligand_smooth, res.x[0],res.x[1],res.x[2]), 'r')
    plt.xlabel(ligand_name+' [M]')
    plt.ylabel('active_probability')
    plt.xscale('log')
    plt.xticks([1e-13,1e-12,1e-11,1e-10,1e-9,1e-8,1e-7,1e-6,1e-5,1e-4])
    #plt.legend("Kd_active={}, Kd_inactive={}, beta_deltaE={}".format(10**res.x[0],10**res.x[1],res.x[2]))
    #plt.figtext(0.2, 0.7, "Kd_active={}, Kd_inactive={}, beta_deltaE={}".format(10**res.x[0],10**res.x[1],res.x[2]), size='large') 

    plt.grid()
    plt.savefig('{}_arr1_fitting.png'.format(ligand_name),dpi=600)  

    """
    p = bokeh.plotting.figure(plot_height=400,
                             plot_width=600,
                             x_axis_label=ligand_name+' [M]',
                              y_axis_label='active_probability',
                              x_axis_type='log')

    p.line(ligand_smooth, agonist_only_p_active_theory(ligand_smooth, res.x[0],res.x[1],res.x[2]))
    p.circle(c, p_active, size=7)


    bokeh.io.show(p);
    """
    res.x = np.around(res.x,2)
    kd_table_arr.loc[len(kd_table_arr)] = [ligand_name,np.around(10**res.x[0]*1e9,2),np.around(10**res.x[1]*1e9,2),np.around(res.x[2],2)]

kd_table_arr

,ligand,Kd_active (nM),Kd_inactive (nM),delta_E
0,BU72,0.10,2.88,-2.55
1,damgo,33.88,245.47,-2.39
2,MP,117.49,257.04,-2.25
3,PZM21,36.31,85.11,-2.48
4,TRV130,32.36,114.82,-2.67
5,buprenorphine,3.39,17.78,-3.29
6,lofentanil,0.02,0.66,-2.57
7,morphine,138.04,436.52,-2.38


In [36]:
kd_table_arr.sort_values(by=['Kd_active (nM)'])

,ligand,Kd_active (nM),Kd_inactive (nM),delta_E
6,lofentanil,0.02,0.66,-2.57
0,BU72,0.10,2.88,-2.55
5,buprenorphine,3.39,17.78,-3.29
4,TRV130,32.36,114.82,-2.67
1,damgo,33.88,245.47,-2.39
3,PZM21,36.31,85.11,-2.48
2,MP,117.49,257.04,-2.25
7,morphine,138.04,436.52,-2.38


In [37]:
kd_table.sort_values(by=['Kd_active (nM)'])

,ligand,Kd_active (nM),Kd_inactive (nM),delta_E
6,lofentanil,0.01,0.12,-1.04
0,BU72,0.07,0.78,-1.08
5,buprenorphine,0.16,0.98,-1.15
2,MP,0.30,3.16,-1.12
7,morphine,0.63,3.16,-1.05
3,PZM21,2.75,13.18,-1.06
1,damgo,2.82,21.38,-1.13
4,TRV,4.68,19.05,-1.10


In [14]:
curdoc().clear()
#initial parameter for optimization
p0 = np.array([-9,-4,-4])
c, p_active = df['conc'].values, df['fentanyl'].values/100

#optimize kd and energy difference to fit the data
res = scipy.optimize.least_squares(resid, p0, args = (c,p_active))

curdoc().clear()
p = bokeh.plotting.figure(plot_height=400,
                         plot_width=600,
                         x_axis_label='[fentanyl] M',
                          y_axis_label='active_probability',
                          x_axis_type='log')

p.line(ligand_smooth, agonist_only_p_active_theory(ligand_smooth, res.x[0],res.x[1],res.x[2]))
p.circle(c, p_active, size=7, color=colors[i])

bokeh.io.show(p);
print(10**res.x)
print(res.x)

[1.50763146e-10 1.78055030e-07 7.27951523e-05]
[-9.82170481 -6.74944575 -4.13789754]


In [67]:
res.x

array([-9.82170484, -6.74944575, -4.1378976 ])

In [11]:
def partial_agonist_only_p_active_theory(c_agonist, log_Kd_active, log_Kd_inactive, beta_deltaE):
    "theoretical curve for mglur5 active probability"
    beta_deltaE = -4
    Kd_active = 10**(log_Kd_active)
    Kd_inactive = 10**(log_Kd_inactive)
    a = (1 + c_agonist/Kd_active)

    b = (1 + c_agonist/Kd_inactive)

    return a/(a+b*np.exp(-beta_deltaE))

def partial_resid(param, c, p_active):
    return p_active - partial_agonist_only_p_active_theory(c, *param)

In [12]:
curdoc().clear()
#initial parameter for optimization
p0 = np.array([-9,-7,-4.5])
c, p_active = df['conc'].values, df['lofentanyl'].values

#optimize kd and energy difference to fit the data
res = scipy.optimize.least_squares(partial_resid, p0, args = (c,p_active))

curdoc().clear()
p = bokeh.plotting.figure(plot_height=400,
                         plot_width=600,
                         x_axis_label='[lofentanyl] M',
                          y_axis_label='active_probability',
                          x_axis_type='log')

p.line(ligand_smooth, partial_agonist_only_p_active_theory(ligand_smooth, res.x[0],res.x[1],res.x[2]))
p.circle(c, p_active, size=7, color=colors[i])

bokeh.io.show(p);
print(10**res.x)
print(res.x)

[1.07989867e-13 1.62414430e-11 1.09647820e-04]
[-12.96661699 -10.78937539  -3.96      ]


In [25]:
curdoc().clear()
#initial parameter for optimization
p0 = np.array([-9,-7,-4])
c, p_active = df['conc'].values, df['PZM21'].values/100

#optimize kd and energy difference to fit the data
res = scipy.optimize.least_squares(partial_resid, p0, args = (c,p_active))

curdoc().clear()
p = bokeh.plotting.figure(plot_height=400,
                         plot_width=600,
                         x_axis_label='[PZM21] M',
                          y_axis_label='active_probability',
                          x_axis_type='log')

p.line(ligand_smooth, partial_agonist_only_p_active_theory(ligand_smooth, res.x[0],res.x[1],res.x[2]))
p.circle(c, p_active, size=7, color=colors[i])

bokeh.io.show(p);
print(10**res.x)
print(res.x)

[4.24527530e-10 2.18645805e-08 1.00000000e-04]
[-9.37209414 -7.66025885 -4.        ]


In [86]:
df = pd.read_excel('./jiawei_trupath_csv/Gi_deer_normalize_linear_terms.xlsx',header=None, names= ["ligand","k", "b"])

plt.figure()
plt.rcParams['font.size'] = '10'
plt.scatter(df['k'], df['b'])
for i in range(len(df)):
    plt.text(df['k'][i], df['b'][i],df['ligand'][i][:-3], )
    #print(df['ligand'][i][:-3])
    #plt.text(df['k'][i], df['b'][i],'yes', )

plt.grid()
plt.savefig('./normalize_linear_terms_cluster',dpi=600)  

In [93]:
df = pd.read_excel('./jiawei_trupath_csv/Gi_TRUPATH_Emax_EC50.xlsx')
df.sort_values(by=['EC50 (nM)'])

,Ligand,Emax,EC50 (nM)
6,BU72,109.50,0.223872
7,Lofentanil,110.40,0.363078
3,Buprenorphine,72.36,0.436516
2,MP,57.31,0.954993
4,Morphine,101.60,1.548817
1,PZM21,81.87,6.606934
5,DAMGO,102.90,8.128305
0,TRV130,78.32,10.715193
